In [1]:
from src.docstore import Docstore

from pathlib import Path
import random

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F


/home/awehrs/miniconda3/envs/rar/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# docstore = Docstore.build(Path("data", "mock"))
# docstore.save(Path("tests", "test_artifacts", "test_docstore"))
docstore = Docstore.load(Path("..", "tests", "test_artifacts", "test_docstore"))


In [3]:
query_idx = random.choices(
    list(range(len(docstore._huggingface_dataset))),
    k=5,
)

In [4]:
queries = docstore._huggingface_dataset[query_idx]["Embeddings"]

In [5]:
queries = np.array(queries).astype(np.float32)

In [6]:
_, examples = docstore.get_nearest_examples(queries, 5)

In [7]:
examples[0]

{'Description': ['Netherlands - Foreign Direct Investment - Net Inflows',
  'Ukraine - GDP Growth Rate',
  'Serbia - Exports',
  'Costa Rica - Government Debt to Gdp',
  'Costa Rica - GDP per capita PPP'],
 'Units': ['EUR Million',
  'percent SA',
  'USD Million',
  'percent of GDP',
  'USD Constant 2017 International Dollars'],
 'Source': ['De Nederlandsche Bank',
  'State Statistics Service Of Ukraine',
  'Statistical Office of the Republic of Serbia',
  'Consejo Monetario Centroamericano',
  'World Bank'],
 'Start_Date': [datetime.date(2003, 6, 30),
  datetime.date(2010, 6, 30),
  datetime.date(2001, 8, 31),
  datetime.date(1991, 12, 31),
  datetime.date(1990, 12, 31)],
 'End_Date': [datetime.date(2021, 12, 31),
  datetime.date(2021, 12, 31),
  datetime.date(2022, 2, 28),
  datetime.date(2020, 12, 31),
  datetime.date(2020, 12, 31)],
 'Publisher': ['SGE', 'SGE', 'SGE', 'SGE', 'SGE'],
 '1960-12-31': [None, None, None, None, None],
 '1961-12-31': [None, None, None, None, None],
 '1962

In [8]:
n_neighbors = 7
search_date = "1/1/2022"
num_samples = 100
resample_freq = "M"
aggregation_fn = np.mean
dropna_threshold = 10
metadata_field = "Embeddings"

_dict = docstore.date_aligned_knn(queries, n_neighbors, search_date, num_samples, resample_freq, aggregation_fn, dropna_threshold, metadata_field)


  1960-12-31 1961-12-31 1962-12-31 1963-12-31 1964-12-31 1965-12-31  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   
2        NaN        NaN        NaN        NaN        NaN        NaN   
3        NaN        NaN        NaN        NaN        NaN        NaN   
4        NaN        NaN        NaN        NaN        NaN        NaN   
5        NaN        NaN        NaN        NaN        NaN        NaN   
6  2856.5269  2729.1516  2847.1514  2879.0193  2896.0085   3075.448   
0       None       None       None       None       None       None   
1       None       None       None       None       None       None   
2       None       None       None       None       None       None   
3       None       None       None       None       None       None   
4       None       None       None       None       None       None   
5       None       None       None       None       None       None   
6     

In [11]:
_dict["metadata"].shape

torch.Size([5, 7, 768])

In [27]:
y = torch.from_numpy(x)
y.shape

torch.Size([7, 100])

In [36]:
z = F.pad(torch.from_numpy(x), (0, 0, 0, 2))

In [40]:
len(_dict["data"][1].columns) - _dict["data"][4].loc[2].isnull().sum() < dropna_threshold

False

In [32]:
data_tensors = []
for df in _dict["data"]:
    data_tensors.append(torch.stack([torch.FloatTensor(x) for x in _dict["data"][2].values]))

torch.stack(data_tensors).shape

torch.Size([5, 7, 100])

In [13]:
len(_dict["data"][0].columns)

79

In [9]:
dfs = [pd.DataFrame(ex)[docstore.time_index] for ex in examples]

df = pd.concat(dfs)

In [10]:
df.columns = pd.to_datetime(df.columns, format="%Y-%m-%d")
df

,1960-12-31,1961-12-31,1962-12-31,1963-12-31,1964-12-31,1965-12-31,1966-12-31,1967-12-31,1968-12-31,1969-12-31,...,2011-12-31,2012-12-31,2013-12-31,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31,2020-12-31
0,None,None,None,None,None,None,None,None,None,None,...,102.590,106.350,105.86,106.03,106.360,104.300,102.800,107.000,110.800,108.70
1,None,None,None,None,None,None,None,None,None,None,...,101.900,104.100,104.80,105.50,105.800,101.100,101.500,102.600,102.400,105.50
2,None,None,None,None,None,None,None,None,None,None,...,1.800,-1.800,4.00,0.90,0.700,0.700,6.500,-2.000,-5.400,0.90
3,None,None,None,None,None,None,None,None,None,None,...,109.200,113.900,111.10,103.60,99.500,108.800,113.300,118.300,118.200,115.70
4,None,None,None,None,None,None,None,None,None,None,...,134.900,140.900,141.40,143.60,146.700,155.300,158.400,160.000,163.000,165.20
0,None,None,None,None,None,None,None,None,None,None,...,5.500,5.100,9.01,10.40,9.272,15.036,16.351,13.619,14.526,15.20
1,None,None,None,None,None,None,None,None,None,None,...,0.600,-0.800,2.40,-4.10,1.400,1.900,0.500,1.100,0.000,0.80
2,None,None,None,None,None,None,None,None,None,None,...,3.500,3.750,NaN,4.75,4.250,NaN,5.000,NaN,NaN,6.25
3,None,None,None,None,None,None,None,None,None,None,...,10.000,11.000,9.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,None,None,None,None,None,None,None,None,...,NaN,7612.390,9552.16,9805.55,10743.010,NaN,NaN,NaN,NaN,NaN


In [55]:
df_resample = df.resample("M", axis=1).agg("mean").ffill(axis=1)
df_resample

,1960-12-31,1961-01-31,1961-02-28,1961-03-31,1961-04-30,1961-05-31,1961-06-30,1961-07-31,1961-08-31,1961-09-30,...,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.9000,69.9000,69.9000,69.9000,69.9000,69.9000,69.9000,69.9000,69.9000,69.6000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8000
2,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,...,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,635.1200
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000,9.0000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-48079.2000,-105906.6000
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,-137.9000
1,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,...,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,660.0700,635.1200
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,6.2500
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,15.9400,15.9400,15.9400,15.9400,15.9400,15.9400,15.9400,15.9400,15.9400,16.4200
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8000


In [ ]:
df = df.fillna(value="NaN")

In [58]:
slice_ = df_resample.loc[:, :"1962-03-15"]
slice_

,1960-12-31,1961-01-31,1961-02-28,1961-03-31,1961-04-30,1961-05-31,1961-06-30,1961-07-31,1961-08-31,1961-09-30,1961-10-31,1961-11-30,1961-12-31,1962-01-31,1962-02-28
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,692.50,692.50,692.50
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,692.50,692.50,692.50
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.06,3.06,3.06
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
slice_.iloc[:, len(slice_.columns) - 5:].head(5)

,1961-10-31,1961-11-30,1961-12-31,1962-01-31,1962-02-28
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,696.3,696.3,692.5,692.5,692.5
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
